# Scraping

In [1]:
import requests
from bs4 import BeautifulSoup


## scraping titulares

In [22]:
url = "https://www.lavozdegalicia.es/"

response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

list_news = []
for i in soup.find_all(class_="a-min-headline"):
    news_url = i.find("a").get("href")
    news_title = i.text.strip()
    list_news.append([news_title, url+str(news_url)])
    
list_news[0:4]


[['Arranca la campaña de las elecciones gallegas',
  'https://www.lavozdegalicia.es//noticia/elecciones-gallegas-18F/2024/01/31/arranca-carrera-san-caetano/00031706725030968211442.htm'],
 ['La portavoz de los familiares del Villa de Pitanxo: «Los 21 fallecidos pueden descansar en paz, la verdad ha salido a la luz»',
  'https://www.lavozdegalicia.es//noticia/somosmar/2024/02/01/21-fallecidos-pueden-descansar-paz-verdad-salido-luz/0003_202402G1P37992.htm'],
 ['Las protestas de los agricultores alimentan el proteccionismo en la Unión Europea',
  'https://www.lavozdegalicia.es//noticia/somosagro/agricultura/2024/02/01/protestas-agricultores-alimentan-proteccionismo-ue/00031706793056141116724.htm'],
 ['Sánchez dice que «todos los independentistas catalanes van a ser amnistiados porque no son terroristas»',
  'https://www.lavozdegalicia.es//noticia/espana/2024/02/01/sanchezasegura-independentistas-catalanes-van-estar-amnistiados-terroristas/00031706800050816928836.htm']]

In [18]:
url_movies = "https://www.sensacine.com/cines/cine/E0770/"

response = requests.get(url_movies)

soup.find_all(class_="meta-title-link")

soup = BeautifulSoup(response.text, 'html.parser')

list_movies = []
for i in soup.find_all(class_="movie-card-theater"):
    sinopsis = i.find(class_="synopsis").text.replace("\n","")
    movie_title = i.find(class_="meta-title-link").text
    movie_ref = i.find(class_="meta-title-link").get("href")

    list_movies.append([movie_title, "https://www.sensacine.com"+movie_ref, sinopsis])

list_movies

[["Miller's Girl",
  'https://www.sensacine.com/peliculas/pelicula-308316/',
  'Cuando un profesor de escritura creativa comience una complicada relación con su joven alumna, surgirán realidades incómodas y su vínculo más allá de lo académico se irá desdibujando y adquiriendo muchos claros y oscuros.'],
 ['Pobres criaturas',
  'https://www.sensacine.com/peliculas/pelicula-290065/',
  'Película que narra la increíble y fantástica historia de Bella Baxter (Emma Stone), una joven que ha revivido el Doctor Godwin Baxter (Willem Dafoe), un científico brillante aunque poco ortodoxo. Libre de los prejuicios de su época, Bella solo desea aprender y defender la igualdad y la liberación.'],
 ['Un mal día lo tiene cualquiera',
  'https://www.sensacine.com/peliculas/pelicula-314048/',
  'Sonia es una mujer ordenada, responsable, una controladora de manual con el sentido de la norma muy integrada. La noche antes del gran día en que defenderá su tesis y se convertirá en doctora, Sonia se topa con Ma

## LEB ORO 

In [60]:
url_feb = "https://baloncestoenvivo.feb.es/resultados/ligaleboro/1/2023"
response = requests.get(url_feb)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find(id="_ctl0_MainContentPlaceHolderMaster_jornadaDataGrid")

lista = []
temp_lista = []

keys = ['Equipos', 'Resultado', 'Fecha', 'Hora']

for fila in table.find_all('tr')[1:]:  # Skip the header row
    temp_dict = {}
    
    for key, linha in zip(keys, fila.find_all(['td', 'th'])):
        temp_dict[key] = linha.text.strip().replace('\r\n\t\t\t\t\t\t\t', ' ')
    
    lista.append(temp_dict)

lista

[{'Equipos': 'HESTIA MENORCA - ALIMERKA OVIEDO BALONCESTO',
  'Resultado': '90-74',
  'Fecha': '21/01/2024',
  'Hora': '12:00'},
 {'Equipos': 'GRUPO URETA TIZONA BURGOS - ICG FORÇA LLEIDA',
  'Resultado': '72-79',
  'Fecha': '20/01/2024',
  'Hora': '17:00'},
 {'Equipos': 'CACERES PATRIMONIO DE LA HUMANIDAD - GUUK GIPUZKOA BASKET',
  'Resultado': '68-86',
  'Fecha': '19/01/2024',
  'Hora': '20:45'},
 {'Equipos': 'AMICS CASTELLÓ - BALONCESTO FUENLABRADA',
  'Resultado': '77-73',
  'Fecha': '19/01/2024',
  'Hora': '20:30'},
 {'Equipos': 'RIOVERDE CLAVIJO - LONGEVIDA SAN PABLO BURGOS',
  'Resultado': '62-79',
  'Fecha': '21/01/2024',
  'Hora': '12:00'},
 {'Equipos': 'MOVISTAR ESTUDIANTES - REAL BETIS BALONCESTO',
  'Resultado': '93-87',
  'Fecha': '21/01/2024',
  'Hora': '12:30'},
 {'Equipos': 'GRUPO ALEGA CANTABRIA - LEYMA CORUÑA',
  'Resultado': '81-83',
  'Fecha': '20/01/2024',
  'Hora': '19:00'},
 {'Equipos': 'UEMC REAL VALLADOLID BALONCESTO - HLA ALICANTE',
  'Resultado': '69-84',
  '

In [59]:
ladder = soup.find(id="_ctl0_MainContentPlaceHolderMaster_clasificacionDataGrid")

ladder

<table border="0" cellpadding="0" id="_ctl0_MainContentPlaceHolderMaster_clasificacionDataGrid">
<tr>
<th scope="col">Po</th><th scope="col">Equipo</th><th scope="col">PJ</th><th scope="col">PG</th><th scope="col">PP</th><th scope="col">PF</th><th scope="col">PC</th><th scope="col">PT</th><th scope="col">R</th>
</tr><tr>
<td>
							1								
						</td><td>
<a href="https://baloncestoenvivo.feb.es/Equipo.aspx?i=922483" id="_ctl0_MainContentPlaceHolderMaster_clasificacionDataGrid__ctl2_equipoHyperLink">MOVISTAR ESTUDIANTES</a>
</td><td>
							18
						</td><td>
							15
						</td><td>
							3
						</td><td>
							1553
						</td><td>
							1332
						</td><td>
							33
						</td><td>
<span id="_ctl0_MainContentPlaceHolderMaster_clasificacionDataGrid__ctl2_rachaLabel" style="color:Blue;">+4</span>
</td>
</tr><tr>
<td>
							2								
						</td><td>
<a href="https://baloncestoenvivo.feb.es/Equipo.aspx?i=922700" id="_ctl0_MainContentPlaceHolderMaster_clasificacionDataGrid__ctl3